In [25]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pandas as pd

spark1 = SparkSession.builder.appName("hw1").getOrCreate()
filePath = 'household_power_consumption.csv'
# df = spark.read.format("csv").option("header", "true").option("sep", ";").load(filePath)
df = spark1.read.csv(filePath, inferSchema=True, header=True, sep=';')

In [26]:
column_list = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']
# column_list = ['Global_active_power']
df = df[column_list]
df.show(5)

+-------------------+---------------------+-------+----------------+
|Global_active_power|Global_reactive_power|Voltage|Global_intensity|
+-------------------+---------------------+-------+----------------+
|              4.216|                0.418| 234.84|            18.4|
|               5.36|                0.436| 233.63|            23.0|
|              5.374|                0.498| 233.29|            23.0|
|              5.388|                0.502| 233.74|            23.0|
|              3.666|                0.528| 235.68|            15.8|
+-------------------+---------------------+-------+----------------+
only showing top 5 rows



In [27]:
# change dtypes
from pyspark.sql.types import DoubleType
for column in column_list:
    df = df.withColumn(column, df[column].cast('double'))
# print(df.dtypes)


In [28]:
# (1) Output the minimum, maximum, and count of the following columns: ‘global active power’, ‘global reactive power’, ‘voltage’, and ‘global intensity’. 
statistics_list = ['max', 'min', 'count']
max_min_count_list = [[df.agg({column: stat}).first()[0] for stat in statistics_list] for column in column_list]

max_min_count_dict = dict()
for i in range(len(column_list)):
    max_min_count_dict[column_list[i]] = max_min_count_list[i]

max_min_count_df = pd.DataFrame(max_min_count_dict, index=statistics_list)
print(max_min_count_df)

       Global_active_power  Global_reactive_power  Voltage  Global_intensity
max                  5.388                  0.528   235.68              23.0
min                  3.520                  0.418   233.29              15.0
count               10.000                 10.000    10.00              10.0


In [29]:
# (2) Output the mean and standard deviation of these columns.
statistics_list = ['mean', 'std']
mean_std_list = [[df.agg({column: stat}).first()[0] for stat in statistics_list] for column in column_list]

mean_std_dict = dict()
for i in range(len(column_list)):
    mean_std_dict[column_list[i]] = mean_std_list[i]

mean_std_df = pd.DataFrame(mean_std_dict, index=statistics_list)
print(mean_std_df)

      Global_active_power  Global_reactive_power     Voltage  Global_intensity
mean             4.225600               0.496400  234.436000         18.140000
std              0.812879               0.037957    0.821397          3.466731


In [44]:
# (3) Perform min-max normalization on the columns to generate normalized output.
from pyspark.ml.feature import VectorAssembler, StandardScaler, MinMaxScaler

vector_assembler = VectorAssembler(inputCols=[column_list[0]], outputCol='ss_feature')
temp_train = vector_assembler.transform(df)
# temp_train.show(2)

minmax_scaler = MinMaxScaler(inputCol='ss_feature', outputCol=column_list[0]+'_scaled')
train = minmax_scaler.fit(temp_train).transform(temp_train)
# train.show(2)
result_spark_df = train[[column_list[0]+'scaled']]
print(result_spark_df.show())

# print(len(result_spark_df))
# result_spark_df.first()[0]

+-------------------------+
|Global_active_powerscaled|
+-------------------------+
|      [0.372591006423983]|
|     [0.9850107066381159]|
|     [0.9925053533190578]|
|                    [1.0]|
|     [0.0781584582441113]|
|                    [0.0]|
|     [0.09743040685224...|
|     [0.09635974304068...|
|     [0.07922912205567...|
|     [0.07601713062098...|
+-------------------------+

None


In [47]:
# (3) Perform min-max normalization on the columns to generate normalized output.
from pyspark.ml.feature import VectorAssembler, StandardScaler, MinMaxScaler

vector_assembler = VectorAssembler(inputCols=column_list, outputCol='ss_feature')
temp_train = vector_assembler.transform(df)
# temp_train.show(2)

minmax_scaler = MinMaxScaler(inputCol='ss_feature', outputCol='scaled')
train = minmax_scaler.fit(temp_train).transform(temp_train)
# train.show(2)
result_spark_df = train[['scaled']]
print(result_spark_df.show())
result_spark_df = result_spark_df.withColumn("")


+--------------------+
|              scaled|
+--------------------+
|[0.37259100642398...|
|[0.98501070663811...|
|[0.99250535331905...|
|[1.0,0.7636363636...|
|[0.07815845824411...|
|[0.0,0.9454545454...|
|[0.09743040685224...|
|[0.09635974304068...|
|[0.07922912205567...|
|[0.07601713062098...|
+--------------------+

None


In [48]:
attrs = sc.parallelize(column_list, 3).zipWithIndex().collect()
print(attrs)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 373.0 failed 1 times, most recent failure: Lost task 0.0 in stage 373.0 (TID 264) (DESKTOP-ILMQ43T executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.ap